读取数据集，清洗，生成提示词

In [71]:
import json
with open('/home/limengze/chen_dongheng/DeepSeek-Prover-V1.5/datasets/enhanced_questions_formalize_0111_total.jsonl','r') as f:
    train_data = [json.loads(line) for line in f]
# with open('/home/limengze/chen_dongheng/DeepSeek-Prover-V1.5/datasets/enhanced_questions_formalize_1_2.jsonl','r') as f:
#     train_data.extend([json.loads(line) for line in f])


In [72]:
#只要成功的证明
with open('/home/limengze/chen_dongheng/DeepSeek-Prover-V1.5/cdh/enhance_proof.json','r') as f:
    pf_data = json.load(f)


In [73]:
len(pf_data)

92

In [74]:
#只要成功的证明
success_proofs =[]
for data in train_data:
    if data['name'] in pf_data:
        data['proof']=pf_data[data['name']]
        success_proofs.append(data)


In [75]:
len(success_proofs)
# with open('new_dataset/success_proofs.json','w') as f:
#     json.dump(success_proofs,f)

92

In [76]:
#以下是gpt构造器_用于生成提示词
import openai
import json

openai.api_base = "https://api2.aigcbest.top/v1"
clientT = openai.Client()
save_pth='new_dataset/'

class gpt_interface:
    def __init__(self):
        with open("/home/limengze/chen_dongheng/pipeline/new_dataset/history.json",'r') as f:
            self.history=json.load(f)
    def get_response(self,id,question):
        messages=[
            {
                "role": "system",
                "content": '''
You are a math assistant. Please give me a natural language proof for the following question in one line: 
'''
            },
            {
                "role": "user",
                "content": question
            }
        ]
        if question in self.history:
            return id,self.history[question]
        #不在，则调用gpt

        response = clientT.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
        result_str=(response.choices[0].message.content)
        self.history[question]=result_str
        self.save()
        return id,result_str
    
    def save(self):
      with open(save_pth+'history.json','w') as f:
          json.dump(self.history,f)
gpt = gpt_interface()

In [77]:
#以下是多线程构造器
import concurrent.futures
from tqdm import tqdm
def parallel_requests(func,prompts_list,max_process=16):
    Flag = True
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func,id,item) for id,item in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in tqdm(concurrent.futures.as_completed(futures)):
            try:
                id,result = future.result()
                results[id]= result
            except Exception as exc:
                Flag = False
    return Flag,results

In [78]:
# 按顺序打包所有的题目
questions = [item['formal_statement'] for item in success_proofs]

In [79]:
# 并行生成所有的答案
success,prompts = parallel_requests(gpt.get_response,questions)

92it [00:00, 91613.48it/s]


In [80]:
# 删除证明的注释
import re
def remove_proof_comments(proof):
    proof = re.sub(r'/-[\s\S]*?-\/', '', proof,re.DOTALL)
    # 删除 -- 开头的行
    lines = proof.splitlines()
    lines = [line for line in lines if not line.strip().startswith('--')]
    # 删除空行
    lines = [line for line in lines if line.strip()]
    # 重新合成行
    text =  '\n'.join(lines)
    # 删除/-开头，-/结尾的所有字符串
    
    return text



In [82]:
#构造训练集
train_set=[]
for i in range(len(prompts)):
    temp = {}
    temp['instruction'] = 'Complete the following Lean 4 code\n\n```lean4\n%s\n``` '%train_data[i]['formal_statement']
    temp['output'] ='\n--%s\n'%prompts[i]+'\n' + remove_proof_comments(success_proofs[i]['proof'][0])
    train_set.append(temp)
    

In [84]:
with open('2025_1_17_train_pairs2.json','w') as f:
    json.dump(train_set,f)